imports

In [1]:
import sys 
import numpy as np 
from sklearn import svm, datasets
from sklearn.model_selection import GridSearchCV
import matplotlib.pyplot as plt
import torch
import torchvision 
import torchvision.transforms as transforms
from sklearn.decomposition import PCA
from sklearn.pipeline import make_pipeline
from sklearn.preprocessing import MinMaxScaler
import pickle

device=torch.device("cuda:0")

In [2]:

transform = transforms.Compose(
    [transforms.ToTensor()])

trainset = torchvision.datasets.CIFAR10(root='./data', train=True,
                                        download=True, transform=transform)
trainloader = torch.utils.data.DataLoader(trainset, batch_size=trainset.__len__(),
                                          shuffle=True, num_workers=2)

testset = torchvision.datasets.CIFAR10(root='./data', train=False,
                                       download=True, transform=transform)
testloader = torch.utils.data.DataLoader(testset, batch_size=testset.__len__(),
                                         shuffle=False, num_workers=2)
classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

Files already downloaded and verified
Files already downloaded and verified


In [17]:
def process(dataloader,transform_list):
    for data in dataloader:
        x,y=data
    x=x.view(x.shape[0],-1)
    for i in transform_list:
        i.fit(x)
        x=i.transform(x)
    y=y.numpy()
    return x,y
def accuracy(preds,targs,training=False):
    p=torch.from_numpy(preds)
    t=torch.from_numpy(targs)
    correct = (torch.eq(t,p)).sum().item()
    accuracy=100*(correct/len(preds))
    if training is False:
        print('overall testing accuracy: ',accuracy)
    else:
        print('overall training accuracy: ',accuracy)
    return accuracy

In [10]:
components=180
pca=PCA(n_components=components)
scaling = MinMaxScaler(feature_range=(-1,1))
tlist=[]
tlist.append(pca);tlist.append(scaling)
C_range = np.logspace(-2,2,5) #[1.e-02 1.e-01 1.e+00 1.e+01 1.e+02]
gamma_range = np.logspace(-3, 1, 5) #[1.e-03 1.e-02 1.e-01 1.e+00 1.e+01]

In [5]:
traininputs,traintargets=process(trainloader,tlist)


In [11]:
testinputs,testtargets=process(testloader,tlist)

In [6]:
print(type(traininputs));print(type(traintargets))


<class 'numpy.ndarray'>
<class 'numpy.ndarray'>


classifiers = []

for c in C_range:
    for gamma in gamma_range:
        svcrbf=svm.SVC(C=c,kernel='rbf',gamma=gamma)
        svc=make_pipeline(pca,scaling,svcrbf)
        classifiers.append([gamma,c,svc])

In [7]:
svc=svm.SVC()
gs_parameters={'gamma':gamma_range,'C':C_range}
gs=GridSearchCV(svc,gs_parameters)

In [8]:
gs.fit(traininputs,traintargets)

GridSearchCV(cv=None, error_score='raise',
       estimator=SVC(C=1.0, cache_size=200, class_weight=None, coef0=0.0,
  decision_function_shape='ovr', degree=3, gamma='auto', kernel='rbf',
  max_iter=-1, probability=False, random_state=None, shrinking=True,
  tol=0.001, verbose=False),
       fit_params=None, iid=True, n_jobs=1,
       param_grid={'gamma': array([1.e-03, 1.e-02, 1.e-01, 1.e+00, 1.e+01]), 'C': array([1.e-02, 1.e-01, 1.e+00, 1.e+01, 1.e+02])},
       pre_dispatch='2*n_jobs', refit=True, return_train_score='warn',
       scoring=None, verbose=0)

train_accuracies=[]
classif_idx=0;
for j in range(5):
    for data in trainloader:
        traininputs,traintargets=data;
        traininputs=flatten(traininputs)
        classifiers[classif_idx][2]=classifiers[classif_idx][2].fit(traininputs,traintargets)
        train_accuracies.append(accuracy(classifiers[classif_idx][2].predict(traininputs),traintargets,training=True))
        print(classifiers[classif_idx])
        classif_idx+=1

In [12]:
preds=gs.predict(testinputs)

To try: rbf kernel, c/gamma values, pca/svd

classaccuracy=np.zeros(len(classes))
ntargets=testtargets.cpu().numpy()
for i in range(len(ntargets)):
    if TestPredictions[i]==ntargets[i]:
        classaccuracy[ntargets[i]]=classaccuracy[ntargets[i]]+1
classaccuracy=classaccuracy/10
for i in range(len(classes)):
    print('class accuracy:',classes[i],',',classaccuracy[i],'%')

from sklearn.metrics import confusion_matrix
cm=confusion_matrix(TestPredictions,ntargets)
print(classes)
print(cm);type(cm)

testing_accuracies=[]
for data in testloader:
    testinputs, testtargets=data;
    testinputs=flatten(testinputs)
for i in classifiers:
    x=accuracy(i[2].predict(testinputs),testtargets)
    testing_accuracies.append([i,x])



print(testing_accuracies[6])

print(trainset.__len__())

valtrainloader = torch.utils.data.DataLoader(trainset, batch_size=trainset.__len__(),shuffle=True, num_workers=2)
valsvcrbf=svm.SVC(C=.1,gamma=.1,kernel='rbf')
valsvc=make_pipeline(pca,scaling,valsvcrbf)
for data in valtrainloader:
    valinputs,valtargets=data;
    valinputs=flatten(valinputs)
valsvc.fit(valinputs,valtargets)


final_model=[valsvc, accuracy(valsvc.predict(valinputs),valtargets)]

accuracy(valsvc.predict(testinputs),testtargets)

svc=svm.SVC(C=1,gamma=.1,kernel='rbf');
psvc=make_pipeline(pca,scaling,svc)
psvc.fit(valinputs,valtargets)
x=[accuracy(psvc.predict(testinputs),testtargets),psvc]

In [18]:
x=accuracy(preds,testtargets)

overall testing accuracy:  19.66


numpy.ndarray